<a href="https://colab.research.google.com/github/ikyath/M5-Forecasting-Accuracy-Kaggle/blob/master/M5_Forecast_Encoder_Decoder_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#**M5 Forecasting - Accuracy**

**Estimate the unit sales of Walmart retail goods**



How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses. 

The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia conducts cutting-edge forecasting research and provides business forecast training. It helps companies achieve accurate predictions, estimate the levels of uncertainty, avoiding costly mistakes, and apply best forecasting practices. The MOFC is well known for its Makridakis Competitions, the first of which ran in the 1980s.

# Understand Business Problem 
In the challenge, you are predicting item sales at stores in various locations for two 28-day time periods. Information about the data is found in the [M5 Participants Guide](https://mofc.unic.ac.cy/m5-competition/).

In [1]:
pwd


'/content'

Connecting to Personal Google drive and accessing data path

In [2]:
cd /content/drive/My\ Drive/Data\ Science


/content/drive/My Drive/Data Science


Importing necessary libraries 

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from tqdm.notebook import tqdm as tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers import RepeatVector,TimeDistributed
from numpy import array
from keras.models import Sequential, load_model
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Using TensorFlow backend.


Importing the data from all the files

In [0]:
sales = pd.read_csv('sales_train_validation.csv')
calendar = pd.read_csv('calendar.csv')
selling_prices = pd.read_csv('sell_prices.csv')
submission_file = pd.read_csv('sample_submission.csv')

## EDA

Lets see data in each file

In [5]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [6]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
selling_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [8]:
selling_prices['store_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

#Lets look our EDA on one of the store - TX_1

Lets create a dataframe for sales table only for store TX_1 with necessary columns.

In [0]:
sales_melt = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day', value_name='demand')

In [0]:
sales_TX_1 = sales_melt[sales_melt.store_id == "TX_1"]
new_TX_1 = pd.merge(sales_TX_1, calendar, left_on="day", right_on="d", how="left")
new_TX_1 = pd.merge(new_TX_1, selling_prices, left_on=["store_id", "item_id", "wm_yr_wk"],right_on=["store_id", "item_id", "wm_yr_wk"], how="left")
new_TX_1["day_int"] = new_TX_1.day.apply(lambda x: int(x.split("_")[-1]))

For each day

For each day we sum over products sell_price and demand

For each day we count_nonzeros over products sell_price and demand

In [0]:
day_sum = new_TX_1.groupby("day_int")[["sell_price", "demand"]].agg("sum").reset_index()

In [12]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=day_sum.day_int, 
                         y=day_sum.demand,
                         mode="lines",
                         name="demand",
                         ),
              row=1,col=1         
              )

fig.add_trace(go.Scatter(x=day_sum.day_int, 
                         y=day_sum.sell_price,
                         mode="lines",
                         name="sell_price",
                         ),
              row=2,col=1           
              )

fig.update_layout(height=1000, title_text="SUM -> Demand  and Sell_price")
fig.show()

Observation :

From the above sum over product demand we observe that some days are "Zeros" because those are Christmas days, I think in chirstmas day the store was closed. and we observe some patterns over the years.

From the above sum over product sell_price we observe that day-by-day the sells are increasing. at the end its becaming constant.

##Lets take some features from calendar dataframe

Here we are creating a dataframe of 1969 days of all columns from calendar dataframe namely mentioning event types and names across all stores.

In [0]:
days = range(1, 1970)
time_series_columns = [f'd_{i}' for i in days]
transfer_cal = pd.DataFrame(calendar[['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']].values.T, index=['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI'], columns= time_series_columns)
transfer_cal = transfer_cal.fillna(0)
event_name_1_se = transfer_cal.loc['event_name_1'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan).fillna(10)
event_name_2_se = transfer_cal.loc['event_name_2'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan).fillna(10)

Below we can see the data mentioned above :
Example SuperBowl event has influenced all stores

In [14]:
transfer_cal

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,...,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
event_name_1,0,0,0,0,0,0,0,0,SuperBowl,0,0,0,0,0,0,0,ValentinesDay,0,0,0,0,0,0,PresidentsDay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,LentStart,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,MemorialDay,0,0,NBAFinalsStart,0,0,0,0,Ramadan starts,0,0,0,0,0,0,0,0,0,0,0,NBAFinalsEnd
event_type_1,0,0,0,0,0,0,0,0,Sporting,0,0,0,0,0,0,0,Cultural,0,0,0,0,0,0,National,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Religious,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,National,0,0,Sporting,0,0,0,0,Religious,0,0,0,0,0,0,0,0,0,0,0,Sporting
event_name_2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Father's day
event_type_2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Cultural
snap_CA,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
snap_TX,0,0,0,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0,1,...,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,0,0,0
snap_WI,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,1,...,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0,0


Below mentioned function is to transform the above mentioned dataframe into features.

Used LabelEncoder for event_name, event_type and stores

Weekday, month and day are cyclic features which are encoded by referring [here](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/)

In [0]:
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature]) 

    
    
    data['wday_cos'] = np.cos(data['wday']/(7/(2*np.pi)))
    data['wday_sin'] = np.sin(data['wday']/(7/(2*np.pi))) 

    data['month_cos'] = np.cos(data['month']/(12/(2*np.pi)))
    data['month_sin'] = np.sin(data['month']/(12/(2*np.pi)))

    data['day_cos'] = np.cos(data["date"].dt.day/(31/(2*np.pi)))
    data['day_sin'] = np.sin(data["date"].dt.day/(31/(2*np.pi)))

    return data

In [0]:
calendar['date'] = pd.to_datetime(calendar['date'])
calendar = calendar[calendar['date']>= '2015-1-27']  #We are considering less data due to memory constraints
calendar= transform(calendar)
# Attempts to convert events into time series data.


### Lets look into the transformed data of calendar

In [17]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,wday_cos,wday_sin,month_cos,month_sin,day_cos,day_sin
1459,2015-01-27,11452,Tuesday,4,1,2015,d_1460,30,4,1,1,0,0,0,-0.900969,-4.338837e-01,0.866025,0.5,0.688967,-7.247928e-01
1460,2015-01-28,11452,Wednesday,5,1,2015,d_1461,30,4,1,1,0,0,0,-0.222521,-9.749279e-01,0.866025,0.5,0.820763,-5.712682e-01
1461,2015-01-29,11452,Thursday,6,1,2015,d_1462,30,4,1,1,0,0,0,0.623490,-7.818315e-01,0.866025,0.5,0.918958,-3.943559e-01
1462,2015-01-30,11452,Friday,7,1,2015,d_1463,30,4,1,1,0,0,0,1.000000,-1.133108e-15,0.866025,0.5,0.979530,-2.012985e-01
1463,2015-01-31,11501,Saturday,1,1,2015,d_1464,30,4,1,1,0,0,0,0.623490,7.818315e-01,0.866025,0.5,1.000000,-2.449294e-16


## Lets filter the required columns and transform the data for better retrieval in future.

In [18]:
transfer_cal = pd.DataFrame(calendar[['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','day_cos','day_sin','wday_cos','wday_sin','month_cos','month_sin']].values.T,
                            index=['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','day_cos','day_sin','wday_cos','wday_sin','month_cos','month_sin'])
transfer_cal

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509
event_name_1,30.000000,30.000000,30.000000,3.000000e+01,3.000000e+01,26.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01,28.000000,30.000000,22.000000,30.000000,11.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,30.000000,12.000000,30.000000,3.000000e+01,30.000000,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,2.300000e+01,3.000000e+01,3.000000e+01,...,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01,30.000000,30.000000,14.000000,3.000000e+01,3.000000e+01,1.700000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,2.400000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,1.600000e+01
event_type_1,4.000000,4.000000,4.000000,4.000000e+00,4.000000e+00,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000e+00,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000e+00,0.000000,4.000000,1.000000,4.000000,2.000000,4.000000,4.000000e+00,4.000000,4.000000,4.000000,4.000000,2.000000,4.000000,4.000000e+00,4.000000,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,4.000000e+00,4.000000e+00,...,4.000000,4.000000,4.000000e+00,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000e+00,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000e+00,4.000000,4.000000,1.000000,4.000000e+00,4.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00
event_name_2,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
event_type_2,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
snap_CA,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,0.

In [19]:
transfer_cal.shape

(13, 510)

 So far we have 13 features 

#Lets look into Selling prices dataframe for extra features

Lets join the calendar and selling price dataframe based on date columns

In [0]:
price_feature = calendar[['wm_yr_wk','date']].merge(selling_prices, on = ['wm_yr_wk'], how = 'left')
price_feature['id'] = price_feature['item_id']+'_'+price_feature['store_id']+'_validation'


Lets view some values in new dataframe

In [21]:
price_feature.head()

,wm_yr_wk,date,store_id,item_id,sell_price,id
0,11452,2015-01-27,CA_1,HOBBIES_1_001,8.26,HOBBIES_1_001_CA_1_validation
1,11452,2015-01-27,CA_1,HOBBIES_1_002,3.97,HOBBIES_1_002_CA_1_validation
2,11452,2015-01-27,CA_1,HOBBIES_1_003,2.97,HOBBIES_1_003_CA_1_validation
3,11452,2015-01-27,CA_1,HOBBIES_1_004,4.64,HOBBIES_1_004_CA_1_validation
4,11452,2015-01-27,CA_1,HOBBIES_1_005,2.88,HOBBIES_1_005_CA_1_validation


We will convert the dataframe over reduced time which will be helpful in connecting features from various dataframes easily

In [0]:
df = price_feature.pivot('id','date','sell_price')

In [23]:
df.head()

date,2015-01-27,2015-01-28,2015-01-29,2015-01-30,2015-01-31,2015-02-01,2015-02-02,2015-02-03,2015-02-04,2015-02-05,2015-02-06,2015-02-07,2015-02-08,2015-02-09,2015-02-10,2015-02-11,2015-02-12,2015-02-13,2015-02-14,2015-02-15,2015-02-16,2015-02-17,2015-02-18,2015-02-19,2015-02-20,2015-02-21,2015-02-22,2015-02-23,2015-02-24,2015-02-25,2015-02-26,2015-02-27,2015-02-28,2015-03-01,2015-03-02,2015-03-03,2015-03-04,2015-03-05,2015-03-06,2015-03-07,...,2016-05-11,2016-05-12,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,2016-05-31,2016-06-01,2016-06-02,2016-06-03,2016-06-04,2016-06-05,2016-06-06,2016-06-07,2016-06-08,2016-06-09,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_2_validation,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_3_validation,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_CA_4_validation,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
FOODS_1_001_TX_1_validation,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24


In [24]:
price_df = sales.merge(df,on=['id'],how= 'left').iloc[:,-300:]
price_df.index = sales.id
price_df.head()

,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,...,2016-05-11,2016-05-12,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,2016-05-31,2016-06-01,2016-06-02,2016-06-03,2016-06-04,2016-06-05,2016-06-06,2016-06-07,2016-06-08,2016-06-09,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,...,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38
HOBBIES_1_002_CA_1_validation,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,...,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97
HOBBIES_1_003_CA_1_validation,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,...,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97
HOBBIES_1_004_CA_1_validation,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,...,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64
HOBBIES_1_005_CA_1_validation,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,...,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88


Here we get demand values for all products for complete 1913 days

In [0]:
days = range(1, 1913 + 1)
time_series_columns = [f'd_{i}' for i in days]
time_series_data = sales[time_series_columns]  #Get time series data

In [26]:
time_series_data.shape

(30490, 1913)

Lets create a Numpy array which consists of all features and demand(Dependent variable) for all products in last 100 days

In [0]:
X = []   #build a data with two features(salse and event1)
for i in tqdm(range(time_series_data.shape[0])):
    X.append([list(t) for t in zip(transfer_cal.loc['event_name_1'][-(100+28):-(28)],
                                   transfer_cal.loc['event_type_1'][-(100+28):-(28)],
                                   transfer_cal.loc['event_name_2'][-(100+28):-(28)],     
                                   transfer_cal.loc['event_type_2'][-(100+28):-(28)],
                                   transfer_cal.loc['snap_CA'][-(100+28):-(28)],
                                   transfer_cal.loc['snap_TX'][-(100+28):-(28)],
                                   transfer_cal.loc['snap_WI'][-(100+28):-(28)],
                                   transfer_cal.loc['day_sin'][-(100+28):-(28)],
                                   transfer_cal.loc['day_cos'][-(100+28):-(28)],
                                   transfer_cal.loc['wday_sin'][-(100+28):-(28)],
                                   transfer_cal.loc['wday_cos'][-(100+28):-(28)],
                                   transfer_cal.loc['month_sin'][-(100+28):-(28)],
                                   transfer_cal.loc['month_cos'][-(100+28):-(28)],
                                   price_df.iloc[i][-(100+28):-(28)],
                                   time_series_data.iloc[i][-100:])]) 

X = np.asarray(X, dtype=np.float32)

In [0]:
X.shape

We have a numpy array of 30490 Products, 100 time steps data with 14 features and 1 dependent variable

Below is the function to normalize and renormalize the data, we have used min-max normalization

In [0]:
def Normalize(list):
    list = np.array(list)
    low, high = np.percentile(list, [0, 100])
    delta = high - low
    if delta != 0:
        for i in range(0, len(list)):
            list[i] = (list[i]-low)/delta
    return  list,low,high

def ReverseNoramlize(list,low,high):
    delta = high - low
    if delta != 0:
        for i in range(0, len(list)):
            list[i] = list[i]*delta + low
    return list

In [39]:
np.random.seed(7)

 ## we only used the last 84 days for train_data.
if __name__ == '__main__':
    train_n,train_low,train_high = Normalize(X[:,-(28*3):,:])
    X_train = train_n[:,-28*3:-28,:14]
    print(X_train.shape)
    y = train_n[:,-28:,14]  #     
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 15
    n_out_seq_length =28
    num_y = 1
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features-1))
    y = y.reshape((y.shape[0], y.shape[1], 1))
    print(X_train.shape)
    print(y.shape)
    # define model

    model = Sequential()

    
    model.add(LSTM(128, activation='tanh', input_shape=(56, n_features-1),return_sequences=True))
    model.add(LSTM(64, activation='tanh',return_sequences=False))
    model.add(RepeatVector(n_out_seq_length))
    model.add(LSTM(32, activation='tanh',return_sequences=True))
    model.add(LSTM(16, activation='tanh',return_sequences=True))
    model.add(Dropout(0.1))  
    model.add(TimeDistributed(Dense(num_y)))   # num_y means the shape of y,in some problem(like translate), it can be many.
                                                #In that case, you should set the  activation= 'softmax'
    model.compile(optimizer='adam', loss='mse')
    # demonstrate prediction
    model.fit(X_train, y, epochs=20, batch_size=1000)

(30490, 56, 14)
(30490, 56, 14)
(30490, 28, 1)
Epoch 1/20
30490/30490 [==============================] - 12s 378us/step - loss: 3.1373e-04
Epoch 2/20
30490/30490 [==============================] - 9s 290us/step - loss: 2.0919e-04
Epoch 3/20
30490/30490 [==============================] - 9s 298us/step - loss: 2.0537e-04
Epoch 4/20
30490/30490 [==============================] - 9s 293us/step - loss: 2.0490e-04
Epoch 5/20
30490/30490 [==============================] - 9s 292us/step - loss: 2.0463e-04
Epoch 6/20
30490/30490 [==============================] - 9s 292us/step - loss: 2.0445e-04
Epoch 7/20
30490/30490 [==============================] - 9s 292us/step - loss: 2.0451e-04
Epoch 8/20
30490/30490 [==============================] - 9s 294us/step - loss: 2.0445e-04
Epoch 9/20
30490/30490 [==============================] - 9s 288us/step - loss: 2.0443e-04
Epoch 10/20
30490/30490 [==============================] - 9s 291us/step - loss: 2.0423e-04
Epoch 11/20
30490/30490 [================

In [41]:
x_input = array(X_train[:,-56:])
x_input = x_input.reshape((30490, 56, n_features-1))
print(x_input.shape)
#x_input = Normalize2(x_input,train_low,train_high)
yhat = model.predict(x_input[:,-56:], verbose=0)
x_input=np.concatenate((x_input[:,:,13].reshape(x_input.shape[0],x_input.shape[1]),yhat.astype(np.float32).reshape(x_input.shape[0],x_input.shape[1]-28)),axis=1).reshape((x_input.shape[0],x_input.shape[1]+28,1));
#print(yhat)
print(x_input.shape)

(30490, 56, 14)
(30490, 84, 1)


In [0]:
x_input = ReverseNoramlize(x_input,train_low,train_high)
# x_input = np.rint(x_input)

In [46]:
forecast = pd.DataFrame(x_input.reshape(x_input.shape[0],x_input.shape[1])).iloc[:,-28:]
forecast.columns = [f'F{i}' for i in range(1, forecast.shape[1] + 1)]
forecast[forecast < 0] =0
forecast.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,1.018306,1.021447,1.011335,0.999818,0.991536,0.987529,0.987243,0.989598,0.993497,0.998041,1.002583,1.006711,1.010201,1.012966,1.015010,1.016392,1.017201,1.017536,1.017493,1.017164,1.016628,1.015950,1.015184,1.014374,1.013553,1.012745,1.011967,1.011233
1,1.136404,1.293931,1.432064,1.543770,1.628143,1.687643,1.726271,1.748441,1.758339,1.759598,1.755185,1.747401,1.737951,1.728034,1.718448,1.709681,1.701994,1.695486,1.690150,1.685910,1.682652,1.680244,1.678553,1.677449,1.676817,1.676553,1.676567,1.676786
2,1.163452,1.356358,1.528486,1.668473,1.774134,1.848249,1.895858,1.922631,1.933961,1.934520,1.928102,1.917630,1.905249,1.892457,1.880231,1.869159,1.859542,1.851481,1.844942,1.839813,1.835937,1.833136,1.831235,1.830066,1.829477,1.829336,1.829531,1.829967
3,1.118311,1.252176,1.367579,1.460382,1.530533,1.580276,1.612916,1.632025,1.640981,1.642723,1.639663,1.633688,1.626206,1.618220,1.610406,1.603185,1.596792,1.591327,1.586797,1.583153,1.580308,1.578163,1.576612,1.575553,1.574891,1.574543,1.574436,1.574510
4,1.165889,1.361982,1.537175,1.679710,1.787291,1.862725,1.911144,1.938334,1.949794,1.950291,1.943693,1.932980,1.920336,1.907285,1.894822,1.883543,1.873753,1.865551,1.858905,1.853696,1.849764,1.846928,1.845008,1.843833,1.843248,1.843118,1.843329,1.843784


In [0]:
validation_ids = sales['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]

In [0]:
ids = np.concatenate([validation_ids, evaluation_ids])

In [0]:
predictions = pd.DataFrame(ids, columns=['id'])
forecast = pd.concat([forecast]*2).reset_index(drop=True)
predictions = pd.concat([predictions, forecast], axis=1)

In [0]:
predictions.to_csv('submission.csv', index=False)  #Generate the csv file.